Import Library

In [36]:
import pandas as pd

#Deep Learning Module
import tensorflow as tf
from tensorflow.keras.layers import Embedding,Dense,LSTM, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [13]:
df=pd.read_csv('train.csv')

In [14]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [15]:
df.dropna(inplace=True)

In [16]:
X=df.drop('label',axis=1)
y=df['label']

In [17]:
print(X.shape)
print(y.shape)

(16750, 4)
(16750,)


In [18]:
messages=X.copy()

In [19]:
messages['title'][10]

'Obama’s Organizing for Action Partners with Soros-Linked ‘Indivisible’ to Disrupt Trump’s Agenda'

In [20]:
messages.reset_index(inplace=True)

Import NLTK function

In [21]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
# Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus[10:20]

['russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hillari',
 'chuck todd buzzfe donald trump polit favor breitbart',
 'monica lewinski clinton sex scandal set american crime stori',
 'rob reiner trump mental unstabl breitbart']

In [26]:
onehot_repr=[one_hot(words,n=5000)for words in corpus] 
onehot_repr

[[3170, 2903, 1877, 4143, 622, 2528, 2257, 824, 753, 4224],
 [1712, 4109, 4538, 2148, 1610, 1585, 4874],
 [1095, 4506, 4513, 1221],
 [2934, 4782, 3630, 129, 3774, 2473],
 [1486, 1610, 845, 2277, 4901, 3376, 1610, 1586, 1986, 2341],
 [3387,
  814,
  3629,
  2465,
  610,
  4955,
  723,
  2570,
  1468,
  4346,
  4380,
  1459,
  2414,
  2997,
  4874],
 [17, 744, 566, 1298, 1136, 1589, 3529, 2712, 4823, 4839, 3201],
 [2733, 4899, 2737, 2541, 4514, 1828, 4955, 384, 4823, 4839, 3201],
 [940, 4288, 1182, 2488, 1359, 3109, 3942, 3456, 4955, 4194],
 [774, 1362, 1845, 406, 4289, 1457, 3740, 641],
 [3434, 2730, 1696, 3299, 327, 2319, 4735, 2577, 4539, 4159, 308],
 [129, 4148, 622, 3109, 4955, 4514],
 [1525, 804, 1938, 4585, 2083, 916, 1948, 181, 6],
 [1612, 2140, 173, 4662, 3509, 3273, 1347, 4823, 4839, 3201],
 [1258, 489, 1312, 1275, 2526, 4823, 4839, 3201],
 [543, 4455, 2380, 656, 4112, 3589, 4443, 8, 680, 4674],
 [2272, 3427, 4109],
 [1633, 3849, 4241, 3113, 4955, 586, 2846, 4874],
 [4666, 2361

Embedding Representation

In [27]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  824  753 4224]
 [   0    0    0 ... 1610 1585 4874]
 [   0    0    0 ... 4506 4513 1221]
 ...
 [   0    0    0 ... 4823 4839 3201]
 [   0    0    0 ... 4865 2051 2415]
 [   0    0    0 ... 4823 4839 3201]]


In [31]:
print(len(embedded_docs))
print(y.shape)

16750
(16750,)


In [33]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Create DeepLearng Model

In [37]:
voc_size=5000
embedding_vector_features=40

model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))



In [38]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


Training Model

In [39]:
model.fit(X_train,
          y_train,
          validation_data=(X_test,y_test),
          epochs=20,
          batch_size=64)


Epoch 1/20
176/176 [==============================] - 9s 38ms/step - loss: 0.3527 - accuracy: 0.8325 - val_loss: 0.1886 - val_accuracy: 0.9195
Epoch 2/20
176/176 [==============================] - 6s 35ms/step - loss: 0.1462 - accuracy: 0.9401 - val_loss: 0.1899 - val_accuracy: 0.9191
Epoch 3/20
176/176 [==============================] - 6s 35ms/step - loss: 0.1017 - accuracy: 0.9614 - val_loss: 0.1943 - val_accuracy: 0.9215
Epoch 4/20
176/176 [==============================] - 6s 35ms/step - loss: 0.0746 - accuracy: 0.9739 - val_loss: 0.2209 - val_accuracy: 0.9146
Epoch 5/20
176/176 [==============================] - 6s 35ms/step - loss: 0.0470 - accuracy: 0.9844 - val_loss: 0.2681 - val_accuracy: 0.9229
Epoch 6/20
176/176 [==============================] - 6s 35ms/step - loss: 0.0316 - accuracy: 0.9900 - val_loss: 0.3156 - val_accuracy: 0.9206
Epoch 7/20
176/176 [==============================] - 6s 35ms/step - loss: 0.0165 - accuracy: 0.9947 - val_loss: 0.3542 - val_accuracy: 0.9153

Measure Performance

In [40]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [50]:
pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [51]:
confusion_matrix(y_test,pred)

array([[2879,  252],
       [ 236, 2161]])

In [52]:
accuracy_score(y_test,pred)

0.9117221418234442